In [3]:
# Импорт библитек, необходимых для EDA
import numpy as np
import pandas as pd
import plotly.express as px
from scipy import stats
from plotly.subplots import make_subplots
from matplotlib import pyplot as plt
import seaborn as sns
colors = ['#b5de2b', '#6ece58', '#35b779', '#1f9e89', '#26828e', '#31688e', '#3e4989', '#482878', '#440154', '#253494', '#225ea8', '#1d91c0','#41b6c4','#7fcdbb','#c7e9b4','#edf8b1','#ffffd9']

## 1. Очистка датасета

In [4]:
relative_path = 'data/AptamerModifData.csv'
df = pd.read_csv(relative_path)
df.head()

,Article,DOI,Name,Origin sequence,Sequence,"Stability t1/2, h",Environment,"Concentration of envi, %","Temperature, oC",Type,"Melting temperature, oC",Modified,Comments
0,Conversion of RNA Aptamer into Modified DNA Ap...,https://doi.org/10.1021/jacs.9b10460,GLB-A04,aUgAUCAAUcGCCUCAaUUCGACA8GAGGCuCaC,1tAFU1tGAFUFCAAFU1tFCGFCFCFUFCA1tAFUFUFCGAFCA2...,72,frozen human serum,95.0,37.0,DNA,NaN,1,NaN
1,Conversion of RNA Aptamer into Modified DNA Ap...,https://doi.org/10.1021/jacs.9b10460,GLB-G25,aUg6UCAAUcGCCUCAaUUCGACAgGAGGCuCaC,1tAFU1tG2tAFUFCAAFU1tFCGFCFCFUFCA1tAFUFUFCGAFC...,72,frozen human serum,95.0,37.0,DNA,NaN,1,NaN
2,Conversion of RNA Aptamer into Modified DNA Ap...,https://doi.org/10.1021/jacs.9b10460,GLB-A04,aUgAUCAAUcGCCUCAaUUCGACA8GAGGCuCaC,1tAFU1tGAFUFCAAFU1tFCGFCFCFUFCA1tAFUFUFCGAFCA2...,1.89,rat blood,100.0,38.8,DNA,NaN,1,NaN
3,Conversion of RNA Aptamer into Modified DNA Ap...,https://doi.org/10.1021/jacs.9b10460,GLB-A04 PEG,peg-aUgAUCAAUcGCCUCAaUUCGACA8GAGGCuCaC,peg1tAFU1tGAFUFCAAFU1tFCGFCFCFUFCA1tAFUFUFCGAF...,5.09,rat blood,100.0,38.8,DNA,NaN,1,NaN
4,Conversion of RNA Aptamer into Modified DNA Ap...,https://doi.org/10.1021/jacs.9b10460,GLB-G25 PEG,peg-aUg6UCAAUcGCCUCAaUUCGACAgGAGGCuCaC,peg1tAFU1tG2tAFUFCAAFU1tFCGFCFCFUFCA1tAFUFUFCG...,4.23,rat blood,100.0,38.8,DNA,NaN,1,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Article                   160 non-null    object 
 1   DOI                       160 non-null    object 
 2   Name                      183 non-null    object 
 3   Origin sequence           184 non-null    object 
 4   Sequence                  158 non-null    object 
 5   Stability t1/2, h         162 non-null    object 
 6   Environment               184 non-null    object 
 7   Concentration of envi, %  183 non-null    float64
 8   Temperature, oC           183 non-null    float64
 9   Type                      184 non-null    object 
 10  Melting temperature, oC   41 non-null     float64
 11  Modified                  184 non-null    int64  
 12  Comments                  3 non-null      object 
dtypes: float64(3), int64(1), object(9)
memory usage: 18.8+ KB


In [6]:
df = df.drop(columns = ['Article', 'DOI', 'Origin sequence', 'Melting temperature, oC', 'Comments'])

In [7]:
df['Stability t1/2, h'] = pd.to_numeric(df['Stability t1/2, h'], errors='coerce')
df = df.dropna()

## 2. Одномерный анализ данных

In [23]:
def plot_pie(df, column, title, colors):
    """
    Построение круговой диаграммы для указанного столбца DataFrame.

    Parameters:
    df (pd.DataFrame): Исходный DataFrame.
    column (str): Название столбца для построения диаграммы.
    title (str): Заголовок диаграммы.
    colors (list): Список цветов для диаграммы.
    """
    if column not in df.columns:
        raise ValueError(f"Столбец '{column}' не найден в DataFrame")

    value_counts = df[column].value_counts()

    # Построение круговой диаграммы
    fig = px.pie(values=value_counts.values, names=value_counts.index, color_discrete_sequence=colors, title=title)

    # Вывод графика
    fig.show()

def plot_bar(df, column, title, colors):
    """
    Построение стобчатой диаграммы для указанного столбца DataFrame.

    Parameters:
    df (pd.DataFrame): Исходный DataFrame.
    column (str): Название столбца для построения диаграммы.
    title (str): Заголовок диаграммы.
    colors (list): Список цветов для диаграммы.
    """
    if column not in df.columns:
        raise ValueError(f"Столбец '{column}' не найден в DataFrame")

    value_counts = df[column].value_counts()

    # Построение диаграммы
    fig = px.bar(df, x=value_counts.index, y=value_counts.values, 
                color=value_counts.index, color_discrete_sequence=colors, 
                title=title, labels={'x': '', 'y': 'Количество'})
    fig.update_layout(showlegend=False)
    # Вывод графика
    fig.show()

In [24]:
plot_bar(df, 'Environment', "Среды, использовавшиеся для определения стабильности", colors)

In [25]:
plot_bar(df, 'Type', "Типы последовательностей, представленных в датасете", colors)

In [54]:
plot_pie(df, 'Modified', "Соотношение модифицированных и немодифицированных последовательностей", colors)

In [ ]:
# Создание подграфиков
fig = make_subplots(rows=1, cols=2, subplot_titles=("Гистограмма", "Violin Plot"), column_widths=[0.5, 0.5])

# Добавление гистограммы
hist_fig = px.histogram(df, x = "Stability t1/2, h",
         color_discrete_sequence = colors, 
         opacity = 0.8)


fig.add_trace(hist_fig['data'][0], row=1, col=1)

# Добавление box диаграммы
violin_fig = px.violin(df, y="Stability t1/2, h",
        color_discrete_sequence=colors)


fig.add_trace(violin_fig['data'][0], row=1, col=2)

# Настройка макета
fig.update_layout(showlegend=True, title_text="Распределение значений периода полураспада", height=600,  width=1200) 

fig.show()

In [ ]:
# Создание подграфиков
fig = make_subplots(rows=1, cols=2, subplot_titles=("Гистограмма ", "Box Plot"), column_widths=[0.5, 0.5])

# Добавление гистограммы
hist_fig = px.histogram(df, x = "Stability t1/2, h", color = 'Type',
         color_discrete_sequence = colors, 
         opacity = 0.7)


for trace in hist_fig['data']:
    fig.add_trace(trace, row=1, col=1)

# Добавление box диаграммы
box_fig = px.box(df, y="Stability t1/2, h", x="Type", color="Type",
        color_discrete_sequence=colors)

for trace in box_fig['data']:
    fig.add_trace(trace, row=1, col=2)

# Настройка макета
fig.update_layout(barmode='stack', showlegend=True, title_text="Распределение значений периода полураспада среди различных типов последовательностей", height=600,  width=1200) 

fig.show()

In [ ]:
# Создание подграфиков
fig = make_subplots(rows=1, cols=2, subplot_titles=("Гистограмма", "Box Plot"), column_widths=[0.5, 0.5])

# Добавление гистограммы
hist_fig = px.histogram(df, x = "Stability t1/2, h", color = 'Modified',
         color_discrete_sequence = colors, 
         opacity = 0.7)


for trace in hist_fig['data']:
    fig.add_trace(trace, row=1, col=1)

# Добавление box диаграммы
box_fig = px.box(df, y="Stability t1/2, h", x="Modified", color="Modified",
        color_discrete_sequence=colors)

for trace in box_fig['data']:
    fig.add_trace(trace, row=1, col=2)

# Настройка макета
fig.update_layout(barmode='stack', showlegend=True, title_text="Распределение значений периода полураспада среди модифицированных и немодифицированных последовательностей", height=600,  width=1200) 

fig.show()

In [37]:
# Создание подграфиков
fig = make_subplots(rows=1, cols=2, subplot_titles=("Гистограмма ", "Box Plot"), column_widths=[0.5, 0.5])

# Добавление гистограммы
hist_fig = px.histogram(df, x = "Stability t1/2, h", color = 'Environment',
         color_discrete_sequence = colors, category_orders={"Environment": sorted(df['Environment'].unique())},
         opacity = 0.7)


for trace in hist_fig['data']:
    fig.add_trace(trace, row=1, col=1)

# Добавление box диаграммы
counts_dict = df.groupby('Environment').size().to_dict()  # Получаем количество значений для каждой категории в словарь
box_fig = px.box(df, y="Stability t1/2, h", x="Environment", color="Environment",
        color_discrete_sequence=colors, category_orders={"Environment": sorted(df['Environment'].unique())},
        hover_data={'Environment': True})  

# Настраиваем hovertemplate для отображения количества значений
for trace in box_fig['data']:
    trace.hovertemplate = 'Environment: %{x}<br>Count: %{hovertext}<extra></extra>'
    trace.hovertext = [counts_dict[cat] for cat in trace.x]  # Назначаем количество значений для каждого следа
    fig.add_trace(trace, row=1, col=2)  # Добавляем каждый след на график

# Настройка макета
fig.update_layout(barmode='stack', showlegend=True, title_text="Распределение значений периода полураспада среди различных типов последовательностей", height=600,  width=1200) 

fig.show()

In [27]:
import re
from collections import defaultdict
from modifications import modifications_dict

# Регулярное выражение для поиска модификаций
# Мы используем позиционный захват, чтобы найти модификации, которые не являются префиксами других модификаций
modification_pattern = re.compile('|'.join(map(re.escape, modifications_dict.keys())))

# Словарь для подсчета модификаций
modification_count = defaultdict(int)

# Проход по каждой строке в датасете
for seq in df['Sequence']:
    # Нахождение всех модификаций в строке
    found_modifications = modification_pattern.findall(seq)
    
    # Множество для хранения уникальных модификаций в этой строке
    unique_modifications = set()
    
    for mod in found_modifications:
        # Проверяем, добавлена ли модификация в множество
        if mod not in unique_modifications:
            # Если нет, добавляем её и увеличиваем счетчик модификации
            unique_modifications.add(mod)
            modification_count[mod] += 1

# Вывод результатов
for mod, count in modification_count.items():
    print(f"{modifications_dict[mod]}: {count}")

3'-monothio: 5
3'-dithio: 5
5-[N-(1-naphthylmethyl)carboxamide]-2'-deoxyuridine: 4
2'F-ANA: 16
cholesterol: 2
2'-fluoro: 39
inverted 3'-thymine: 38
galactose-polyethylene glycol: 1
3'-thiophosphoryl: 14
methylphosphonate: 1
5-(N-benzylcarboxamide)-2'-deoxyuridine: 1
2'-allyl: 6
chimeric deoxyribonucleotide: 13
2'-methoxy group: 42
2'-methylene: 3
2'-difluoromethylene: 3
2'-amino: 10
polyamine-polyethylene glycol 20 kDa: 2
polyethylene glycol 20 kDa: 2
polyamine-polyethylene glycol 40 kDa: 1
polyethylene glycol 40 kDa: 3
5' thiol: 27
chimeric ribonucleotide: 2
hexaethylene glycol linker: 2
inverted: 3
locked nucleic acid: 5
hexilamine linker from the picture with polyethylene glycol: 2


In [43]:
modification_df = pd.DataFrame.from_dict(modification_count, orient='index', columns=['Count'])
modification_df = modification_df.reset_index()
modification_df['Modification'] = modification_df['index'].map(modifications_dict)

# Создание столбчатой диаграммы
fig = px.bar(
    modification_df, 
    x='Modification', 
    y='Count', 
    color='Modification', 
    color_discrete_sequence=colors, 
    title='Количество модификаций каждого вида',
    labels={'Modification': 'Вид модификации', 'Count': 'Количество вхождений'},
    text='Count'  # Добавляем значения на столбцы
)

fig.update_layout(
    showlegend=False,    
    height=600,  width=1200
)

fig.update_traces(textposition='outside')

# Отображение графика
fig.show()